In [0]:
import torch
from torch.utils import data

import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt

import numpy as np
import pickle

from google.colab import auth

device = "cuda" if torch.cuda.is_available() else "cpu"

In [0]:
!wget -r -N -c -np --user nhulkund --ask-password https://physionet.org/files/picdb/1.0.0/

In [0]:
# Read Data into DF

admissions = pd.read_csv('physionet.org/files/picdb/1.0.0/ADMISSIONS.csv.gz', compression='gzip')
chartevents = pd.read_csv('physionet.org/files/picdb/1.0.0/CHARTEVENTS.csv.gz', compression='gzip')
diagnoses_icd = pd.read_csv('physionet.org/files/picdb/1.0.0/DIAGNOSES_ICD.csv.gz', compression='gzip')
d_icd_diagnoses = pd.read_csv('physionet.org/files/picdb/1.0.0/D_ICD_DIAGNOSES.csv.gz', compression='gzip')
d_items = pd.read_csv('physionet.org/files/picdb/1.0.0/D_ITEMS.csv.gz', compression='gzip')
d_labitems = pd.read_csv('physionet.org/files/picdb/1.0.0/D_LABITEMS.csv.gz', compression='gzip')
emr_symptoms = pd.read_csv('physionet.org/files/picdb/1.0.0/EMR_SYMPTOMS.csv.gz', compression='gzip')
icu_stays = pd.read_csv('physionet.org/files/picdb/1.0.0/ICUSTAYS.csv.gz', compression='gzip')
input_events = pd.read_csv('physionet.org/files/picdb/1.0.0/INPUTEVENTS.csv.gz', compression='gzip')
lab_events = pd.read_csv('physionet.org/files/picdb/1.0.0/LABEVENTS.csv.gz', compression='gzip')
patients = pd.read_csv('physionet.org/files/picdb/1.0.0/PATIENTS.csv.gz', compression='gzip')
prescriptions = pd.read_csv('physionet.org/files/picdb/1.0.0/PRESCRIPTIONS.csv.gz', compression='gzip')
surgery_vital_signs = pd.read_csv('physionet.org/files/picdb/1.0.0/SURGERY_VITAL_SIGNS.csv.gz', compression='gzip')

In [0]:
d_items_df=pd.DataFrame(d_items)
print(d_items['ITEMID'][100:150])
d_items['ITEMID']=d_items['ITEMID'].str.replace(" ","")
print(type(d_items.ITEMID[0]))
print(d_items.loc[d_items['ITEMID']=='5141'])

In [0]:
# Easier to use: 

item_dict = dict() 
for _, row in d_items.iterrows(): 
  item_dict[row.ITEMID] = row.LABEL

lab_item_dict = dict()
for _, row in d_labitems.iterrows(): 
  lab_item_dict[row.ITEMID] = row.LABEL

ICD_CN_TO_ICD = dict() 
for _, row in d_icd_diagnoses.iterrows(): 
  ICD_CN_TO_ICD[row.ICD10_CODE_CN] = row.ICD10_CODE 


Here we include only the first admission of each patient.

In [0]:
# Clean: Include only the first admission

admissions = admissions.sort_values(by = ['ADMITTIME'])

admits_to_keep = []
seen_patients = set()

for _, row in admissions.iterrows(): 
  if row.SUBJECT_ID not in seen_patients: 
    admits_to_keep.append(row.HADM_ID)
    seen_patients.add(row.SUBJECT_ID)

In [0]:
def remove_admits(df): 
  return df[df['HADM_ID'].isin(admits_to_keep)]

admissions = remove_admits(admissions)
chartevents = remove_admits(chartevents)
diagnoses_icd = remove_admits(diagnoses_icd)
emr_symptoms = remove_admits(emr_symptoms)
icu_stays = remove_admits(icu_stays)
input_events = remove_admits(input_events)
lab_events = remove_admits(lab_events)
prescriptions = remove_admits(prescriptions)
surgery_vital_signs = remove_admits(surgery_vital_signs)


Helper functions to parse admit times.

In [0]:
from datetime import date, timedelta, time, datetime

def to_datetime(x): 
  li = x.split()
  my_date = li[0].split("-")
  my_time = li[1].split(":")

  ret = datetime(int(my_date[0]), int(my_date[1]), int(my_date[2]), int(my_time[0]), int(my_time[1]), int(my_time[2]))
  
  return ret

age_at_admission = dict()  
birth_date = dict()
admit_date = dict() 
for _, row in patients.iterrows(): 
  birth_date[row.SUBJECT_ID] = to_datetime(row.DOB)

for _, row in admissions.iterrows(): 
  admit_date[row.SUBJECT_ID] = to_datetime(row.ADMITTIME)
  age_at_admission[row.SUBJECT_ID] = to_datetime(row.ADMITTIME) - birth_date[row.SUBJECT_ID]

In [0]:
# Time since admission (hours)
def normalize_time(patient_id, x): 
  delta = to_datetime(x) - admit_date[patient_id]
  return delta.total_seconds() / 3600.0 

In [0]:
patient_set = set([p for p in patients.SUBJECT_ID])

In [0]:
chartevents['HOURS_IN'] = chartevents.apply(lambda row: normalize_time(row.SUBJECT_ID, row.CHARTTIME), axis=1)
lab_events['HOURS_IN'] = lab_events.apply(lambda row: normalize_time(row.SUBJECT_ID, row.CHARTTIME), axis=1)
surgery_vital_signs['HOURS_IN'] = surgery_vital_signs.apply(lambda row: normalize_time(row.SUBJECT_ID, row.MONITORTIME), axis=1)

In [0]:
import math 
## Feature Set

## Chart Features
chart_feats = [1001, 1002, 1003, 1004, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016]


# Surgery Vital Signs
surgery_feats = surgery_vital_signs['ITEMID'].value_counts().index.tolist() 


lab_feats = [5225, 
             5097, 
             #5141, 
             5129, 
             5257, 
             5114,
             5113,
             5115,
             5132,
             5136,
             5226,
             5230,
             5218,
             5224,
             5212,
             5033,
             5041,
             5223,
             5215,
             5174,
             5111,
             #6317,
             #5094,
             #5492,
             #5002,
             5075,
             5237,
             5249,
             5235,
             5239,
             5227,
             5026,
             5031,
             5024,
             6085
             ]



In [0]:
for id in lab_feats:
  print(id,lab_item_dict[id])

In [0]:
def get_feature_name(idx): 
  if idx < (len(lab_feats)): 
    return lab_item_dict[lab_feats[idx]]
  elif idx < (len(lab_feats) + len(chart_feats)): 
    return item_dict[str(chart_feats[idx - len(lab_feats)])]
  elif idx < (len(lab_feats) + len(chart_feats) + len(surgery_feats)): 
    return item_dict[surgery_feats[idx - len(lab_feats) - len(chart_feats)]]
  elif idx < (len(lab_feats) + len(chart_feats) + len(surgery_feats) + 2):
    return 'gender'
  else: 
    return 'age'

def get_feature_name_flattened(idx): 
  hours_in = idx // (len(lab_feats) + len(chart_feats) + len(surgery_feats))

  idx -= hours_in * (len(lab_feats) + len(chart_feats) + len(surgery_feats))

  if hours_in == WINDOW_SIZE: 
    if idx < 2: 
      return 'gender'
    else: 
      return 'age'
  else: 
    if idx < (len(lab_feats)): 
      return lab_item_dict[lab_feats[idx]]
    elif idx < (len(lab_feats) + len(chart_feats)): 
      return item_dict[str(chart_feats[idx - len(lab_feats)])]
    elif idx < (len(lab_feats) + len(chart_feats) + len(surgery_feats)): 
      return item_dict[surgery_feats[idx - len(lab_feats) - len(chart_feats)]]

In [0]:
def key_fn(tup):
  return abs(tup[0])

def sort_importance(coefficients, feat_name_fn):
  coef_shape = coefficients.shape
  print(coef_shape)
  importance = []

  for i in range(coef_shape[1]):
    importance.append((coefficients[0,i], feat_name_fn(i)))
    # print('index {} gives {}'.format(i, feat_name_fn(i)))
  
  return sorted(importance, key=key_fn, reverse=True)

In [0]:
def convert_float_tensor(X):
  return torch.tensor(X).float()

def perturb_features(model, X, feature_range=None):
  if feature_range is None:
    feature_range = (0, X.shape[2])

  print(feature_range)
  perturb_effects = []
  tensor_x = convert_float_tensor(X)
  orig_out = model(tensor_x)

  for ind in range(feature_range[0], feature_range[1]):
    variable_name = get_feature_name(ind)
    # print(f'Dealing with variable {variable_name}')
    perturbation = np.random.normal(0.0, 0.2, size=X.shape[:2])
    X[:, :, ind] = X[:, :, ind] + perturbation
    effect = ((orig_out - model(convert_float_tensor(X))) ** 2).mean() ** 0.5
    print(f'Variable {ind+1} name ({variable_name}), perturbation effect: {effect:.4f}')
    perturb_effects.append((effect, variable_name, ind))
    X[:, :, ind] = X[:, :, ind] - perturbation
  
  return sorted(perturb_effects, key=key_fn, reverse=True)

We use these to index into the tensors that follow (i.e. chart_X[patient_index_of[subject_id]] is what you want, not chart_X[subject_id]. Similar for item_id's

In [0]:
# More Helper Dicts
chart_index_of = dict() 
for i in range(len(chart_feats)): 
  chart_index_of[chart_feats[i]] = i
  
lab_index_of = dict() 
for i in range(len(lab_feats)): 
  lab_index_of[lab_feats[i]] = i

surgery_index_of = dict() 
for i in range(len(surgery_feats)): 
  surgery_index_of[surgery_feats[i]] = i


print(chart_index_of)
print(lab_index_of)
print(surgery_index_of)

patient_index_of = dict() 
cc = 0
for p in patient_set: 
  patient_index_of[p] = cc 
  cc += 1
  
  

# Feature / Label Definition Section

Lets only sample up to 100 hours. That seems reasonable, the whole stay of ~75% of patients are captured here.

In [0]:
MAX_HOURS = 100
WINDOW_SIZE = 24
GAP_TIME = 6
PRED_SIZE = 6

In [0]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

Here, **feat_val** can be either min, max, or mean depending on the task you want. Change the code beneath it to fit the task. Remember to change the loop to go over lab events of the ID you want. Here, it is the **minimum wbc count** measured over an hour-long window.

**feat_cnt** is used to make sure the measurement is made over the prediction interval.

**feat_agg** is the label we use in prediction. feat_val[ i ] [ j ] denotes the lab test value for patient i where our window starts at hour j. In other words, it is the aggregate value over hours [j + WINDOW_SIZE + GAP_SIZE, j + WINDOW_SIZE + GAP_TIME + PRED_SIZE). Here, I'm again computing the min. You should change this if you want max/mean/whatever.

**window_cnt** is the number of times the test was taken in the desired window. The window is [j + WINDOW_SIZE + GAP_SIZE, j + WINDOW_SIZE + GAP_TIME + PRED_SIZE)

In [0]:
subjects_to_remove = set() 

feat_agg = np.zeros((len(patient_set), MAX_HOURS))
window_cnt = np.zeros((len(patient_set), MAX_HOURS))

feat_val = np.zeros((len(patient_set), MAX_HOURS))
feat_cnt = np.zeros((len(patient_set), MAX_HOURS))

###
def aggregate(x, type):
  if type == 'MIN': 
    return np.min(x)
  elif type == 'MAX': 
    return np.max(x)
  else: 
    return np.mean(x)
###

###
for _, row in lab_events[lab_events['ITEMID'] == 5141][lab_events['HOURS_IN'] < MAX_HOURS].iterrows():
  if row.HOURS_IN < 0: 
    subjects_to_remove.add(row.SUBJECT_ID) 
  
  elif is_number(row.VALUE): 
    my_idx = patient_index_of[row.SUBJECT_ID]
    my_bkt = int(row.HOURS_IN)

    ### 
    if feat_cnt[my_idx][my_bkt] == 0: 
      feat_val[my_idx][my_bkt] = row.VALUENUM
    else: 
      feat_val[my_idx][my_bkt] = np.amin([feat_val[my_idx][my_bkt], row.VALUENUM])
    ###

    feat_cnt[my_idx][my_bkt] += 1
###

###
for i in range(len(patient_set)): 
  for j in range(MAX_HOURS - WINDOW_SIZE - GAP_TIME - PRED_SIZE):
    li = [] 
    for k in range(j+WINDOW_SIZE+GAP_TIME, j+WINDOW_SIZE+GAP_TIME+PRED_SIZE): 
      window_cnt[i][j] += feat_cnt[i][k]
      if feat_cnt[i][k] > 0: 
        li.append(feat_val[i][k])
    if len(li) > 0: 
      feat_agg[i][j] = aggregate(np.array(li), type = 'MIN')
###

This is just normal stuff.

In [0]:
chart_X = np.zeros((len(patient_set), MAX_HOURS, len(chart_feats)))
chart_Xcnt = np.zeros((len(patient_set), MAX_HOURS, len(chart_feats)))
lab_X = np.zeros((len(patient_set), MAX_HOURS, len(lab_feats)))
lab_Xcnt = np.zeros((len(patient_set), MAX_HOURS, len(lab_feats)))
surgery_X = np.zeros((len(patient_set), MAX_HOURS, len(surgery_feats)))
surgery_Xcnt = np.zeros((len(patient_set), MAX_HOURS, len(surgery_feats)))

for _, row in lab_events[lab_events['HOURS_IN'] < MAX_HOURS][lab_events['ITEMID'].isin(lab_feats)].iterrows():
  if row.HOURS_IN < 0: 
    subjects_to_remove.add(row.SUBJECT_ID)
  elif is_number(row.VALUE): 
    lab_X[patient_index_of[row.SUBJECT_ID]][int(row.HOURS_IN)][lab_index_of[row.ITEMID]] += row.VALUENUM
    lab_Xcnt[patient_index_of[row.SUBJECT_ID]][int(row.HOURS_IN)][lab_index_of[row.ITEMID]] += 1 

for _, row in surgery_vital_signs[surgery_vital_signs['HOURS_IN'] < MAX_HOURS][surgery_vital_signs['ITEMID'].isin(surgery_feats)].iterrows():
  if row.HOURS_IN < 0: 
    subjects_to_remove.add(row.SUBJECT_ID)
  elif is_number(row.VALUE): 
    surgery_X[patient_index_of[row.SUBJECT_ID]][int(row.HOURS_IN)][surgery_index_of[row.ITEMID]] += row.VALUE
    surgery_Xcnt[patient_index_of[row.SUBJECT_ID]][int(row.HOURS_IN)][surgery_index_of[row.ITEMID]] += 1 

for _, row in chartevents[chartevents['HOURS_IN'] < MAX_HOURS][chartevents['ITEMID'].isin(chart_feats)].iterrows():
  if row.HOURS_IN < 0: 
    subjects_to_remove.add(row.SUBJECT_ID)
    continue 
  elif is_number(row.VALUE): 
    chart_X[patient_index_of[row.SUBJECT_ID]][int(row.HOURS_IN)][chart_index_of[row.ITEMID]] += row.VALUENUM 
    chart_Xcnt[patient_index_of[row.SUBJECT_ID]][int(row.HOURS_IN)][chart_index_of[row.ITEMID]] += 1 




Here I have simple Forward/Backward Imputation implemented. If time, we can try to implement the various other ones mentioned by https://www.nature.com/articles/s41598-018-24271-9 

global_mean is the mean of each feature over all time points and all patients. If a patient has no occurances of a feature at any time point, it's replaced by the global mean. Otherwise, we propagate values forward/backward to replace missing values. 

In [0]:
# Missing Data Imputation

# Forward/Backward Imputation

# Compute Global means first. 

global_chart_mean = np.zeros(len(chart_feats))
global_lab_mean = np.zeros(len(lab_feats))
global_surgery_mean = np.zeros(len(surgery_feats))

for k in range(len(chart_feats)): 
  global_chart_mean[k] = np.sum(chart_X[:, :, k]) / np.sum(chart_Xcnt[:, :, k])
for k in range(len(lab_feats)): 
  global_lab_mean[k] = np.sum(lab_X[:, :, k]) / np.sum(lab_Xcnt[:, :, k])
for k in range(len(surgery_feats)):
  global_surgery_mean[k] = np.sum(surgery_X[:, :, k]) / np.sum(surgery_Xcnt[:, :, k])


def forward_backward_impute(feats, global_mean): 
  # INPUTS: 
  # Feats -- (MAX_HOURS, num_feats)
  # glboal_mean -- (num_feats)
  # OUTPUTS: 
  # ret -- (MAX_HOURS, num_feats) (imputed)
  ret = feats 
  for j in range(feats.shape[1]):
    for i in range(1, MAX_HOURS): 
      if ret[i][j] <= 0: 
        ret[i][j] = ret[i-1][j]
    for i in range(MAX_HOURS-2, -1, -1): 
      if ret[i][j] <= 0: 
        ret[i][j] = ret[i+1][j]
    for i in range(MAX_HOURS): 
      if ret[i][j] <= 0: 
        ret[i][j] = global_mean[j]
  return ret 



In [0]:
# Set up X, Y 


# Set up labels

patient_set = list(patient_set)

gender_one_hot = np.zeros((len(patient_set), 2))
age_vec = np.zeros((len(patient_set), 1))
for _, row in patients.iterrows(): 
  if row.SUBJECT_ID in patient_set: 
    age_vec[patient_index_of[row.SUBJECT_ID]][0] = (age_at_admission[row.SUBJECT_ID].total_seconds() / 3600.0)
    if row.GENDER == 'M': 
      gender_one_hot[patient_index_of[row.SUBJECT_ID]][0] = 1
    else: 
      gender_one_hot[patient_index_of[row.SUBJECT_ID]][1] = 1

static_vec = np.concatenate((gender_one_hot, age_vec), axis = 1)
# [num_patients, 3]

chart_vec = chart_X / (chart_Xcnt + (chart_Xcnt == 0))
lab_vec = lab_X / (lab_Xcnt + (lab_Xcnt == 0))
surgery_vec = surgery_X / (surgery_Xcnt + (surgery_Xcnt == 0))

for i in range(len(patient_set)): 
  chart_vec[i] = forward_backward_impute(chart_vec[i], global_chart_mean)
  lab_vec[i] = forward_backward_impute(lab_vec[i], global_lab_mean)
  surgery_vec[i] = forward_backward_impute(surgery_vec[i],  global_surgery_mean)

time_vec = np.concatenate((lab_vec, chart_vec, surgery_vec), axis=2)
# time_vec [num_patients, max_hours, num_lab_features + num_chart_features + num_vital_features]

# concatenate this with static_vec [num_patients, 3]

**cohort** is a list of indices that you're training / testing on. For instance, if I want the patients with ID's 5, 10, 6, then cohort [patient_index_of[5], patient_index_of[10], patient_index_of[6]].

If you want to test a certain age cohort, then **cohort** should be the a list of indices of patients in that age cohort. 

Ok here I'm going to set up a DataLoader for the tasks.

In [0]:
from torch.utils.data import DataLoader, Dataset

BATCH_SIZE = 16

class MyDataset(Dataset): 
  def __init__(self, feats, values, statics): 
    self.feats = feats # features you're ingesting
    self.values = values # value aggregate in the prediction window
    self.statics = statics # demographic features corresponding to this guy
  def __len__(self): 
    return self.feats.size(0)
  def __getitem__(self, index): 
    return self.feats[index], self.values[index], self.statics[index]

def get_dataloader(indices, model_type, test = False): 
  my_feats = []
  my_values = []
  my_statics = []
  for i in indices:   
    for t_start in range(MAX_HOURS - WINDOW_SIZE - GAP_TIME - PRED_SIZE): 
      if window_cnt[i][t_start] > 0: 
        my_feats.append(time_vec[i][t_start:t_start+WINDOW_SIZE])
        my_values.append(feat_agg[i][t_start])
        my_statics.append(static_vec[i])
  if model_type == 'LSTM': 
    if test: 
      return DataLoader(MyDataset(torch.tensor(my_feats).float(), torch.tensor(my_values).float(), torch.tensor(my_statics).float()), 
                      batch_size=1, 
                      shuffle=True, 
                      drop_last=True)
    else: 
      return DataLoader(MyDataset(torch.tensor(my_feats).float(), torch.tensor(my_values).float(), torch.tensor(my_statics).float()), 
                      batch_size=BATCH_SIZE, 
                      shuffle=True, 
                      drop_last=True)
  else: 
    return my_feats, my_values, my_statics


Here, we're going to set up a cohort to test. Call this prior to train_rf or train_lstm.

In [0]:
def set_up_cohort(cohort): 
  global time_vec
  global static_vec
  msk = np.array([False for i in range(len(time_vec))])
  for p in cohort: 
    msk[p] = True 
  scaler1 = StandardScaler()
  scaler2 = StandardScaler() 
  time_vec = np.array(time_vec)
  static_vec = np.array(static_vec)
  shp = time_vec[msk, ...].shape 
  time_vec[msk, ...] = scaler1.fit_transform(time_vec[msk, ...].reshape(-1, shp[-1])).reshape(shp)
  static_vec[msk, ...] = scaler2.fit_transform(static_vec[msk, ...])


Now we have our RF helper functions.

In [0]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def train_rf(task, cohort, n_estimators=200, bootstrap=True, max_features='sqrt'): 
  train_indices, test_indices = train_test_split(np.array(cohort), test_size=0.2)

  train_data = get_dataloader(train_indices, 'RF')
  test_data = get_dataloader(test_indices, 'RF')
  
  x_train = np.array(train_data[0])
  x_train = x_train.reshape((x_train.shape[0], x_train.shape[1] * x_train.shape[2]))
  x_train = np.concatenate((x_train, np.array(train_data[2])), axis=1)
  ### Obviously change this.
  print(len(np.array(train_data[1])))
  print("max",np.array(train_data[1]).max())
  print("min",np.array(train_data[1]).min())
  print("mean",np.array(train_data[1]).mean())
  print(np.array(train_data[1]))
  y_train = np.int_(np.array(train_data[1]) < 4)
  ###
  x_test = np.array(test_data[0])
  x_test = x_test.reshape((x_test.shape[0], x_test.shape[1] * x_test.shape[2]))
  x_test = np.concatenate((x_test, np.array(test_data[2])), axis=1)

  ### And this.
  y_test = np.int_(np.array(test_data[1]) < 4)
  ###

  model = RandomForestClassifier(n_estimators=n_estimators, 
                             bootstrap = bootstrap,
                             max_features = max_features)
  
  model.fit(x_train, y_train)

  return model, x_test, y_test

def evaluate_rf(model, x_test, y_test): 

  rf_predictions = model.predict(x_test)
  print(model.predict_proba(x_test))
  rf_probs = model.predict_proba(x_test)[:, 1]
  print(rf_probs)
  auc = roc_auc_score(y_test, rf_probs)
  acc = np.sum(rf_predictions == y_test) / len(y_test)

  return auc, acc, rf_predictions, rf_probs 
 

In [0]:
from sklearn.metrics import roc_curve
def plot_roc(title, labels, probs): 
  fpr, tpr, thresholds = roc_curve(labels, probs) 
  plt.figure()
  plt.plot(fpr, tpr, label=title)
  plt.plot([0, 1], [0, 1],'r--')
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('1 - Specificity')
  plt.ylabel('Sensitivity')
  plt.title('ROC')
  plt.legend(loc="lower right")
  plt.show()

Note we need to further preprocess this data (zero mean, unit variance, PCA, etc..)


RF Helper Funcs

In [0]:
model_filename = 'AUC_rf_cohort4_WBC.sav'
effect_filename='RF_model_cohort4_WBC.pt'
LSTM_filename='LSTM_model_cohort4_WBC.pt'
def get_mask(remove_these): 
  msk = [True for i in range(len(patient_set))]
  for p in remove_these: 
    msk[patient_index_of[p]] = False
  return msk 

def run_task_rf(task): 
  
  cohort = [i for i in range(len(patient_set))]
  mask = get_mask(subjects_to_remove)
  cohort = np.array(cohort)[mask, ...]

  COHORTS = ['Total', '0 - 2 Month', '2 Month - 2 Years', '2 Years - 5 Years', '5 Years - 12 Years']
  THRESHOLDS = [-1, 60 * 24, 2 * 365 * 24, 5 * 365 * 24, 12 *  365 * 24]  

  min_age=THRESHOLDS[2]
  max_age=THRESHOLDS[3]

  my_msk = [True for i in range(len(cohort))]
  for i in range(len(cohort)): 
    age_here = static_vec[cohort[i]][-1]
    if age_here < min_age: 
      my_msk[i] = False
    elif age_here > max_age: 
      my_msk[i] = False 
  
  cohort = cohort[my_msk, ...]

  set_up_cohort(cohort)
  model, x_test, y_test = train_rf(task, cohort)

  auc, acc, _, _ = evaluate_rf(model, x_test, y_test)

  return auc, acc, model



In [0]:
auc, acc, model_to_save = run_task_rf('Lab Test Prediction')
print("AUC of ", auc)
print("ACC of ", acc)
 

In [0]:
print(age_at_admission[12879])

In [0]:
import pickle
#filename = 'WBC_all_AUC_80.sav'
from google.colab import drive
#drive.mount('gdrive')
from google.colab import files
#files.download(pickle.dump(model))
#from google.colab import files
pickle.dump(model_to_save, open(model_filename, 'wb'))
files.download(model_filename)

# Change this function. Here I'm getting the input X and label Y from the dataloader features.

Currently, I'm defining the labels as WBC < 20.

In [0]:
TRAIN_BATCHES_PER_EPOCH = 1000
TEST_SAMPLES = 2000

In [0]:
def get_XY(feats, values, statics):
  # feats is B x WINDOW x dim_input
  # values is B
  # statics is B x 3
  X = torch.cat((torch.tensor(feats), torch.tensor(statics).unsqueeze(1).expand((len(feats), len(feats[0]), len(statics[-1])))), dim=-1)
  Y = torch.tensor(torch.tensor(values) < 4).float()

  return X, Y


In [0]:
class LSTM_Classifier(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers=1, dropout=0., bidirectional=False):
    super(LSTM_Classifier, self).__init__()

    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers 
    self.bidirectional = bidirectional
    self.dropout = dropout

    self.rnn = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True,
                      dropout=dropout, bidirectional=bidirectional)
    self.out = nn.Linear(hidden_size + hidden_size * int(bidirectional), 1)

  def forward(self, input):
    # Input is (B, seq_len, input_size)
    rnn_out, _ = self.rnn(input)
    # rnn_out is (B, seq_len, directions * hidden_size)
    # output is (B, seq_len, 1)
    return self.out(rnn_out)

def rnn_train_one_sample(model, criterion, rnn_optimizer, sent_tensor, tag_tensor, alpha = 0.5, clip=None):

    # sent_tensor is (B, Num Hours, Num feats)
    # tag_tensor is (B)

    model.zero_grad() 

    outputs = model(sent_tensor).squeeze(2)

    # loss = criterion(outputs, tag_tensor) * alpha + criterion(outputs[-1], tag_tensor[-1]) * (1.0-alpha)
    loss = criterion(outputs[:, -1], tag_tensor) 

    loss.backward()

    if clip != None: 
      torch.nn.utils.clip_grad_norm(model.parameters(), max_norm=clip)

    rnn_optimizer.step()

    return outputs, loss.item()


In [0]:
import time
import math
import sklearn
from sklearn.metrics import precision_recall_fscore_support

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def evaluate_result(true_tag_list, predicted_tag_list, probs):
  return np.mean(true_tag_list.numpy() == predicted_tag_list), roc_auc_score(true_tag_list, probs)

# Make prediction for one sentence.
def rnn_predict_one_sent(model, sent_tensor):
  
    outputs = model(sent_tensor).squeeze(2).squeeze(0)
    prob = torch.sigmoid(outputs[-1])

    predicted_tag_id = 0
    if prob > 0.5: 
      predicted_tag_id = 1
    
    return predicted_tag_id, prob.item()


def evaluate_rnn(model, test_dataloader): 
  
  model.eval()
  predicted_tags = []
  probs = []
  labels = []

  iter_count = 0
  for feats, values, statics in test_dataloader:
    iter_count += 1
    if iter_count > TEST_SAMPLES: 
      break 

    sent_tensor, tag_tensor = get_XY(feats, values, statics) 
    sent_tensor = sent_tensor.to(device)
    predicted_tag_id, prob = rnn_predict_one_sent(model, sent_tensor)
    predicted_tags.append(predicted_tag_id)
    probs.append(prob)
    labels.append(tag_tensor[0].item())


  acc, auc = evaluate_result(torch.tensor(labels), predicted_tags, probs)

  return auc, acc, predicted_tags, probs 

def train_model(model, criterion, optimizer, train_dataloader, test_dataloader, n_epochs=5, print_every=100, plot_every=50, learning_rate=1e-3, alpha = 0.5, clip=None): 

  iter_count = 0

  current_loss = 0
  current_norm = 0
  all_losses = []
  all_norms = []

  start = time.time()

  model.train()
  for epoch_i in range(n_epochs):
    num_batches = 0
    for feats, values, statics in train_dataloader: 
      num_batches += 1
      if num_batches > TRAIN_BATCHES_PER_EPOCH: 
        break 

      sent_tensor, tag_tensor = get_XY(feats, values, statics)
    
      sent_tensor = sent_tensor.to(device)
      tag_tensor = tag_tensor.to(device)
  
      output, loss = rnn_train_one_sample(model, criterion, optimizer, sent_tensor, tag_tensor, alpha=alpha, clip=clip)
      current_loss += loss

      if iter_count % print_every == 0:
          print('%d %s %.4f' % (iter_count, timeSince(start), current_loss / print_every))
          current_loss = 0

      iter_count += 1

    auc, acc, _, _ = evaluate_rnn(model, test_dataloader)
    print("Epoch ", epoch_i, " ACC of ", acc, " AUC of ", auc)
  return all_losses, all_norms


In [0]:
def train_rnn(task, cohort, n_epochs=10, 
              rnn_clip = 1.0, 
              rnn_hidden_size = 64, 
              rnn_num_layers = 2, 
              learning_rate = 1e-4, 
              rnn_dropout = 0.5, 
              rnn_alpha = 0.5, 
              weight_decay = 1e-4,
              rnn_bidirectional = False): 

  num_feats = time_vec.shape[-1] + 3

  train_indices, test_indices = train_test_split(np.array(cohort), test_size=0.2)

  train_dataloader = get_dataloader(train_indices, 'LSTM')
  test_dataloader = get_dataloader(test_indices, 'LSTM', test=True)

  rnn_model = LSTM_Classifier(input_size=num_feats, hidden_size=rnn_hidden_size, num_layers=rnn_num_layers, dropout=rnn_dropout, bidirectional=rnn_bidirectional)
  criterion = nn.BCEWithLogitsLoss()
  rnn_optimizer = torch.optim.Adam(rnn_model.parameters(), lr=learning_rate, weight_decay=weight_decay)

  losses, norms = train_model(rnn_model, criterion, rnn_optimizer, train_dataloader, test_dataloader, n_epochs=n_epochs, alpha=rnn_alpha, clip=rnn_clip)

  return rnn_model, test_dataloader

In [0]:
# Evaluation

def get_mask(remove_these): 
  msk = [True for i in range(len(patient_set))]
  for p in remove_these: 
    msk[patient_index_of[p]] = False
  return msk 

def run_task_rnn(task): 

  cohort = [i for i in range(len(patient_set))]
  mask = get_mask(subjects_to_remove)
  cohort = np.array(cohort)[mask, ...]
  set_up_cohort(cohort)
  model, test_dataloader = train_rnn(task, cohort)

  auc, acc, _, _ = evaluate_rnn(model, test_dataloader)

  return model, auc, acc, test_dataloader

 

Set up data and evaluate.

In [0]:
model, auc, acc, test_dataloader = run_task_rnn('LAB TEST PREDICTION')
print("AUC of ", auc)
print("ACC of ", acc)


In [0]:
xt = torch.zeros((TEST_SAMPLES, WINDOW_SIZE, time_vec.shape[-1] + 3))

torch.save(model,effect_filename)
files.download(effect_filename)

torch.save(model,LSTM_filename)
files.download(LSTM_filename)

cnt = 0
for feats, values, statics in test_dataloader:
    cnt += 1
    if cnt > TEST_SAMPLES: 
      break 
    sent_tensor, tag_tensor = get_XY(feats, values, statics) 
    xt[cnt-1] = sent_tensor.squeeze(0)
torch.save(xt,'xt.pt')

In [0]:
torch.save(xt,'xt.pt')

#torch.load('xt.pt')

effects = perturb_features(model, xt)
print(effects)
for t in effects:
  print(t[1])

In [0]:

torch.save(effects,effect_filename)
files.download(effect_filename)

In [0]:
COHORTS = ['Total', '0 - 2 Month', '2 Month - 2 Years', '2 Years - 5 Years', '5 Years - 12 Years']
THRESHOLDS = [-1, 60 * 24, 2 * 365 * 24, 5 * 365 * 24, 12 *  365 * 24]